In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("salalah_allocation.csv", parse_dates=["date"])

In [1]:
df.head(20)

NameError: name 'df' is not defined

In [4]:
df["group"] = (df["allocation"] != df["allocation"].shift()).cumsum()

In [5]:
# Create a list to store the result
result_data = []

# Iterate over the groups and calculate start_date and end_date
for group_id, group_data in df.groupby("group"):
    start_date = group_data["date"].iloc[0]
    end_date = group_data["date"].iloc[-1]
    allocation = group_data["allocation"].iloc[0]
    result_data.append(
        {"start_date": start_date, "end_date": end_date, "allocation": allocation}
    )

# Drop the 'group' column (optional)
df.drop("group", axis=1, inplace=True)

# Convert the result_data list into a DataFrame
result_df = pd.DataFrame(result_data)

print(result_df)

   start_date   end_date  allocation
0  2023-11-01 2023-11-04          10
1  2023-11-05 2023-11-08          15
2  2023-11-09 2023-11-14          22
3  2023-11-15 2023-11-29          15
4  2023-11-30 2023-12-12          20
5  2023-12-13 2023-12-22          15
6  2023-12-23 2024-01-03          10
7  2024-01-04 2024-01-10          15
8  2024-01-11 2024-01-12          22
9  2024-01-13 2024-01-24          15
10 2024-01-25 2024-02-02          22
11 2024-02-03 2024-02-10          10
12 2024-02-11 2024-03-05          15
13 2024-03-06 2024-04-13          10
14 2024-04-14 2024-04-23          22
15 2024-04-24 2024-04-26          15
16 2024-04-27 2024-04-30          10


In [2]:
import json
from datetime import datetime

output = []

for _, row in result_df.iterrows():
    dict = {
        "model": "reports.roominventory",
        "fields": {
            "room": 6761,
            "start_date": row["start_date"].strftime("%Y-%m-%d"),
            "end_date": row["end_date"].strftime("%Y-%m-%d"),
            "quantity": row["allocation"],
            "freesale": False,
            "charter": False,
            "operator_group": 2,
        },
    }
    output.append(dict)

output_json = json.dumps(output, indent=2, default=str)
with open("roominventory.json", "w") as f:
    f.write(output_json)

NameError: name 'result_df' is not defined

In [ ]:
records = df.to_dict(orient="records")

with Session(engine) as session:
    for record in records:
        try:
            stmt = insert(reservations_booking).values(record)
            session.execute(stmt)
            session.commit()
        except exc.IntegrityError as e:
            session.rollback()
            if isinstance(e.orig, UniqueViolation):
                record["flag"] = "amendment"
            elif isinstance(e.orig, ForeignKeyViolation):
                record["flag"] = "not mapped"
            stmt = (
                insert(reservations_booking_staging)
                .values(record)
                .on_conflict_do_update(
                    constraint="reservations_booking_staging_pkey",
                    set_=dict(stmt.excluded),
                )
            )
            session.execute(stmt)
            session.commit()